In [1]:
import cv2
import math
import numpy as np

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse

## Auxiliary Functions

In [2]:
def showImage(image, name):
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def convImgToBinary(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img_binary

In [4]:
def CompareResult(BlendingResult, S2, metric):
    
    if metric == 1:
        # Signal to noise ratio, units: decibals
        ErrorValue = cv2.PSNR(S2,BlendingResult)
        print("PSNR: ", ErrorValue,"dB")
        pass
    elif metric == 2:
        # mean structural similarity index over the image, unit: index between 0 & 1
        ErrorValue = ssim(S2, BlendingResult, multichannel=True)
        print("SSIM: ", ErrorValue)
        pass
    elif metric == 3:
        # mean-squared error between two images
        ErrorValue = mse(S2, BlendingResult)
        print("MSE: ", ErrorValue)
        pass
    else:
        ErrorValue = "No Metric Specified"
        pass
    
    return ErrorValue

## Functions

In [5]:
def inpaintTalea(img, mask, inpaintRadius = 25):
    inpaintImg = cv2.inpaint(img, mask, inpaintRadius, cv2.INPAINT_TELEA)
    return inpaintImg

In [6]:
def inpaintNS(img, mask, inpaintRadius = 25):
    inpaintImg = cv2.inpaint(img, mask, inpaintRadius, cv2.INPAINT_NS)
    return inpaintImg

## Runner

In [7]:
S1 = cv2.imread("COTSDataset-master/Part 2 - Multiple Objects/lp_book_no/1_colour.jpeg", 1)
S2 = cv2.imread("COTSDataset-master/Part 2 - Multiple Objects/lp_book_no/2_colour.jpeg", 1)
ObjectMask = cv2.imread("COTSDataset-master/Part 2 - Multiple Objects/lp_book_no/masks/lpbooks_no_3_colour_mask_2_mask.png", 1)
ObjectMask = convImgToBinary(ObjectMask)

In [13]:
# tista tbiddel inpaintRadius billi zzid prameter
# inpaintResultTalea = inpaintTalea(S2, ObjectMask, 500)
inpaintResultTalea = inpaintTalea(S2, ObjectMask, 500)
showImage(inpaintResultTalea, "Inpaint Image")
for metric in range(1,4):
    errorValue = CompareResult(inpaintResultTalea, S1, metric)

PSNR:  34.95550254202244 dB
SSIM:  0.8896322828636848
MSE:  20.77447735821759


In [10]:
inpaintResultNS = inpaintNS(S2, ObjectMask)
showImage(inpaintResultNS, "Inpaint Image")
for metric in range(1,4):
    errorValue = CompareResult(inpaintResultNS, S1, metric)

PSNR:  31.620668159766637 dB
SSIM:  0.8853093711840078
MSE:  44.77272677951389
